# Evaluation for gradient boosting + vit feature extraction

Removed all modifications to the train set except those necessary for feature engineering.

References:

- https://www.kaggle.com/code/abdmental01/multimodel-isic
- https://www.kaggle.com/code/nosherwantahir/notebookea3cca46ba

In [1]:
%load_ext memory_profiler

import io
import warnings
import gc
import sys
import os
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
from tqdm import tqdm

import h5py
from PIL import Image
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow

from scipy.special import softmax

from sklearn.model_selection import *
from sklearn.preprocessing import *
from sklearn.metrics import *

import lightgbm as lgb
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from xgboost import XGBClassifier

import torch
from torchvision.transforms import v2 as transforms
from transformers import AutoModelForImageClassification
from torch.utils.data import Dataset, DataLoader

pd.set_option('display.max_columns', None)

from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import OrdinalEncoder
import category_encoders as ce

import joblib

OWN_INSTANCE = True
SEED = 42
n_splits = 3

os.makedirs('gradboost', exist_ok = True)

# Load and preprocess metadata

In [2]:
%%time 

test_metadata_file = '/kaggle/input/isic-2024-challenge/test-metadata.csv'
train_metadata_file = '/kaggle/input/isic-2024-challenge/train-metadata.csv'

if OWN_INSTANCE:
    test_metadata_file = 'data/test-metadata.csv'
    train_metadata_file = 'data/train-metadata.csv'

test = pd.read_csv(test_metadata_file)
train = pd.read_csv(train_metadata_file)

#train.drop('isic_id',axis=1,inplace=True)
#test.drop('isic_id',axis=1,inplace=True)

test_columns = set(test.columns)
train_columns = set(train.columns)

diff_test_train = test_columns - train_columns
diff_train_test = train_columns - test_columns

if not diff_test_train and not diff_train_test:
    print("Both DataFrames have the same columns.")
else:
    print("Columns present in test but not in train:", diff_test_train)
    print("Columns present in train but not in test:", diff_train_test)

train.drop(columns=['iddx_4', 'mel_mitotic_index', 'iddx_1', 'lesion_id', 'tbp_lv_dnn_lesion_confidence',
                    'iddx_5', 'mel_thick_mm', 'iddx_2', 'iddx_full', 'iddx_3'],inplace=True)

Columns present in test but not in train: set()
Columns present in train but not in test: {'iddx_full', 'mel_mitotic_index', 'iddx_4', 'lesion_id', 'iddx_5', 'tbp_lv_dnn_lesion_confidence', 'mel_thick_mm', 'iddx_2', 'target', 'iddx_3', 'iddx_1'}
CPU times: user 2.42 s, sys: 306 ms, total: 2.72 s
Wall time: 2.72 s


In [3]:
%%time

def fe(df):
    
    # a sort of eccentricity
    df["lesion_size_ratio"]=df["tbp_lv_minorAxisMM"]/df["clin_size_long_diam_mm"]
    # another dimensionless measure of eccentricity (think circle / square)
    df["lesion_shape_index"]=df["tbp_lv_areaMM2"]/(df["tbp_lv_perimeterMM"]**2)
    # contrast between hue inside and outside
    df["hue_contrast"]= (df["tbp_lv_H"]-df["tbp_lv_Hext"]).abs()
    # contrast between luminance inside and outside
    df["luminance_contrast"]= (df["tbp_lv_L"]-df["tbp_lv_Lext"]).abs()
    # LAB is another color space similar to RGB. delta's are inside v. outside.
    df["lesion_color_difference"]=np.sqrt(df["tbp_lv_deltaA"]**2+df["tbp_lv_deltaB"]**2+df["tbp_lv_deltaL"]**2)
    # both metrics increase when asymmetry is higher and are on scale 0-10
    df["border_complexity"]=df["tbp_lv_norm_border"]+df["tbp_lv_symm_2axis"]
    # position on 3D TBP
    df["3d_position_distance"]=np.sqrt(df["tbp_lv_x"]**2+df["tbp_lv_y"]**2+df["tbp_lv_z"]**2)
    # another measure of irregularity...?
    df["perimeter_to_area_ratio"]=df["tbp_lv_perimeterMM"]/df["tbp_lv_areaMM2"]
    # contrast between lesion and surrounding, values from 5-25 + color variation 0 - 10
    df["lesion_visibility_score"]=df["tbp_lv_deltaLBnorm"]+df["tbp_lv_norm_color"]
    # both are location indicators
    df["combined_anatomical_site"]=df["anatom_site_general"]+"_"+df["tbp_lv_location"]
    # only when both are large does a lesion score high on this (cf border_complexity)
    df["symmetry_border_consistency"]=df["tbp_lv_symm_2axis"]*df["tbp_lv_norm_border"]
    # whether the variation in color is similar inside and outside lesion
    df["color_consistency"]=df["tbp_lv_stdL"]/df["tbp_lv_stdLExt"]
    # interactions are just products
    df["size_age_interaction"]=df["clin_size_long_diam_mm"]*df["age_approx"]
    # hue inside and color irregularity
    df["hue_color_std_interaction"]=df["tbp_lv_H"]*df["tbp_lv_color_std_mean"]
    # three measures of irregularity combined.
    df["lesion_severity_index"]=(df["tbp_lv_norm_border"]+df["tbp_lv_norm_color"]+df["tbp_lv_eccentricity"])/3
    df["shape_complexity_index"]=df["border_complexity"]+df["lesion_shape_index"]
    # first three terms are average contrast, last term is contrast in immediately surrounding skin
    df["color_contrast_index"]=df["tbp_lv_deltaA"]+df["tbp_lv_deltaB"]+df["tbp_lv_deltaL"]+df["tbp_lv_deltaLBnorm"]
    # the malignant lesions can be way longer and a log scale might better capture this
    df["log_lesion_area"]=np.log(df["tbp_lv_areaMM2"]+1)
    # perhaps lesion gorws in size with age.
    df["normalized_lesion_size"]=df["clin_size_long_diam_mm"]/df["age_approx"]
    # internal and external hue averaged
    df["mean_hue_difference"]=(df["tbp_lv_H"]+df["tbp_lv_Hext"])/2
    # combining inner contrast assuming Gaussisna
    df["std_dev_contrast"]=np.sqrt((df["tbp_lv_deltaA"]**2+df["tbp_lv_deltaB"]**2+df["tbp_lv_deltaL"]**2)/3)
    # combine metrics of color and shape, both could be more irregular for malignant
    df["color_shape_composite_index"]=(df["tbp_lv_color_std_mean"]+df["tbp_lv_area_perim_ratio"]+df["tbp_lv_symm_2axis"])/3
    df["3d_lesion_orientation"]=np.arctan2(df["tbp_lv_y"],df["tbp_lv_x"])
    df["overall_color_difference"]=(df["tbp_lv_deltaA"]+df["tbp_lv_deltaB"]+df["tbp_lv_deltaL"])/3
    df["symmetry_perimeter_interaction"]=df["tbp_lv_symm_2axis"]*df["tbp_lv_perimeterMM"]
    # the larger this value, the larger the "irregularity"
    df["comprehensive_lesion_index"]=(df["tbp_lv_area_perim_ratio"]+df["tbp_lv_eccentricity"]+df["tbp_lv_norm_color"]+df["tbp_lv_symm_2axis"])/4
    
    # categorical columns
    n_cat = ["combined_anatomical_site"]
    
    return df, n_cat

train, n_cat = fe(train)
test, _ = fe(test)

# columns with categories
cat_cols = ["sex", "tbp_tile_type", "tbp_lv_location", "tbp_lv_location_simple",'patient_id',
   'anatom_site_general','copyright_license','attribution','image_type'] + n_cat

# drop columns only present in one set
def align_columns(train, test):
    common_cols = train.columns.intersection(test.columns)
    train = train[common_cols]
    test = test[common_cols]
    return train, test

# target will be removed by align_columns anyway, remove first and add back later.
target = train['target']
train_features = train.drop(columns=['target'], errors='ignore')

train_features_aligned, test_features_aligned = align_columns(train_features, test)

encoder = ce.OrdinalEncoder(cols=cat_cols, handle_unknown='ignore')
train = encoder.fit_transform(train_features_aligned)
# a second call to encoder.transform will apply the same statistics of fit_transform.
test = encoder.transform(test_features_aligned)

train['target'] = target

train.drop(columns = ['isic_id'], inplace = True)
test.drop(columns = ['isic_id'], inplace = True)

CPU times: user 1.7 s, sys: 443 ms, total: 2.14 s
Wall time: 2.14 s


# Load ViT and extract feature from last hidden layer

## Preparation for image dataset

In [4]:
val_transform = transforms.Compose(
    [
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]
)

model_path = '/kaggle/input/vit/transformers/default/1/'
hdf_test_path = '/kaggle/input/isic-2024-challenge/test-image.hdf5'
hdf_train_path = '/kaggle/input/isic-2024-challenge/train-image.hdf5'

if OWN_INSTANCE:
    model_path = 'TobanDjan/vit'
    hdf_test_path = 'data/test-image.hdf5'
    hdf_train_path = 'data/train-image.hdf5'

# Function to load images from encoded data
def load_image_from_encoded_data(encoded_data):
    image = Image.open(io.BytesIO(encoded_data))
    return image.convert('RGB')

# Define a custom Dataset for the HDF5 images
class HDF5TestDataset(Dataset):
    def __init__(self, image_data, ids, transform=None):
        self.image_data = image_data
        self.ids = ids
        self.transform = transform

    def __len__(self):
        return len(self.image_data)

    def __getitem__(self, idx):
        image_data = self.image_data[idx]
        image = load_image_from_encoded_data(image_data)
        #imshow(image)
        #plt.show()
        if self.transform:
            image = self.transform(image)
        
        # print(image.element_size() * image.nelement())
        # 602112 B = 0.574 MB
        return image, self.ids[idx]

def get_dataset(hdf_file_path):
    with h5py.File(hdf_file_path, 'r') as f:
        image_data = [f[image_id][()] for image_id in tqdm(f.keys())]
        ids = list(f.keys())
        dataset = HDF5TestDataset(image_data=image_data, ids=ids, transform=val_transform)
    
    return dataset

# %memit train_dataset = get_dataset(hdf_train_path)
test_dataset = get_dataset(hdf_test_path)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00, 1971.01it/s]


In [5]:
# Create the test dataset and dataloader
batch_size = 2 ** 8

if OWN_INSTANCE:
    batch_size = 2 ** 8

test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=4)
# train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False, num_workers=4)

model = None
gc.collect()
with torch.no_grad():
    torch.cuda.empty_cache()

device = torch.device("cuda")
model = AutoModelForImageClassification.from_pretrained(model_path)
model.to(device)

def combine_vit_features(dataloader, metadata):
    NUM_FEATURES = 768
    feat_cols = list(range(NUM_FEATURES))

    table = metadata.values
    table = np.hstack((table, np.zeros((table.shape[0], NUM_FEATURES + 1))))
    columns = metadata.columns.values.tolist()
    columns.append('vit_target')
    columns.extend(feat_cols)

    row_offset = 0
    
    model.eval()
    with torch.no_grad():
        for inputs, batch_ids in tqdm(dataloader, total = len(dataloader)):
            inputs = inputs.to(device)
            # print(inputs.element_size() * inputs.nelement())
            outputs = model(inputs, output_hidden_states = True)

            proba = outputs.logits.cpu()
            proba = softmax(proba, axis=1)[:, 1]

            last_hidden_states = outputs.hidden_states[-1].cpu()
            cls_features = last_hidden_states[:, 0, :].squeeze().numpy().tolist()

            for i, isic_id in enumerate(batch_ids):
                row_idx = row_offset + i
                table[row_idx, -NUM_FEATURES - 1] = proba[i]
                table[row_idx, -NUM_FEATURES:] = cls_features[i]

            row_offset += len(batch_ids)

    # metadata.reset_index(inplace = True)
    return pd.DataFrame(table, columns = columns)

%memit test = combine_vit_features(test_dataloader, test)
# %memit train = combine_vit_features(train_dataloader, train)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  2.85it/s]

peak memory: 1671.04 MiB, increment: 73.95 MiB


In [6]:
def pauc_above_tpr(solution, submission, min_tpr: float=0.80):
    v_gt = abs(np.asarray(solution)-1)
    v_pred = np.array([1.0 - x for x in submission])
    max_fpr = abs(1-min_tpr)
    partial_auc_scaled = roc_auc_score(v_gt, v_pred, max_fpr=max_fpr)
    partial_auc = 0.5 * max_fpr**2 + (max_fpr - 0.5 * max_fpr**2) / (1.0 - 0.5) * (partial_auc_scaled - 0.5)
    return partial_auc


# Gradient boosting

In [7]:
def Predict_GB(models, test_data):
    # k-fold cross-validation
    test_predictions = []

    for model in models:
        y_test_pred_proba = model.predict_proba(test_data)[:, 1]
        test_predictions.append(y_test_pred_proba)

    return test_predictions

# TODO: set directory for Kaggle
def load_models(framework:str):
    models = [] 
    for idx in range(3):
        model_file = ""
        if OWN_INSTANCE:
            model_file = f"gradboost/{framework}_{idx}.joblib"
        model = joblib.load(model_file)
        models.append(model)

    return models


all_models = load_models("lgbm")
Cat_all_models = load_models("catboost")
xgb_all_models = load_models("xgb")

%memit test_preds = Predict_GB(all_models, test)
%memit cat_test_preds = Predict_GB(Cat_all_models, test)
%memit xgb_test_preds = Predict_GB(xgb_all_models, test)

peak memory: 1725.26 MiB, increment: 5.60 MiB
peak memory: 1728.86 MiB, increment: 3.65 MiB
peak memory: 1731.18 MiB, increment: 2.31 MiB


# Test

In [8]:
%%time

sample_file = '/kaggle/input/isic-2024-challenge/sample_submission.csv'

if OWN_INSTANCE:
    sample_file = 'data/sample_submission.csv'
    
Sample = pd.read_csv(sample_file)

lgb_test = np.mean(test_preds, axis=0)
cat_test = np.mean(cat_test_preds, axis=0)
xgb_test = np.mean(xgb_test_preds, axis=0)

ensemble_preds = (lgb_test + cat_test + xgb_test) / 3

sub = pd.DataFrame({
    'isic_id': Sample['isic_id'],
    'target': ensemble_preds
})

sub.to_csv('submission.csv', index=False)
sub.head()


CPU times: user 5.78 ms, sys: 0 ns, total: 5.78 ms
Wall time: 4.8 ms


,isic_id,target
0,ISIC_0015657,0.000025
1,ISIC_0015729,0.000012
2,ISIC_0015740,0.000018
